# Bachelor 2021

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import style 
style.use('ggplot')
from scipy.stats import norm
import scipy

In [30]:
S0 = 100
K = 100
T = 1
r = 0.07
sigma = 0.2
n_simulations = 100000
n_steps = 252

In [31]:
class Asian_option_pricing(object):
    def __init__(self, S0, K, T, r, sigma, n_simulations, n_steps):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.n_simulations = n_simulations
        self.n_steps = n_steps   
    
    @property
    def MC_Payoff(self):
        # Udvikling i prisen på det underliggende aktiv  
        dt = self.T/self.n_steps
        x = np.random.normal(0,1,size=(self.n_simulations, self.n_steps))
        S = np.zeros((self.n_simulations, self.n_steps))
        S[:,0] += self.S0
        for i in range(1, n_steps):
            S[:,i] += S[:,i-1]*np.exp((self.r-(self.sigma**2)/2)*dt+self.sigma*np.sqrt(dt)*x[:,i])
        
        def Geo_Mean_Overflow(iterable):
            temp = np.log(iterable)
            return np.exp(temp.mean()) 
        
        ### Formel kommer fra Paul Glasserman, 2004, s. 94
        ### Her skal evt. udvides med flere/andre processer ift. Geometrisk Brownian Motions
        
        # Gennemsnit af prisen på det underliggende aktiv per simulering
        avg = [Geo_Mean_Overflow(S[i,:]) for i in range(len(S))]
#         avg = []
#         for i in range(len(S)):
#             avg.append(Geo_Mean_Overflow(S[i,:]))
        
        # Beregning af payoff for call
        c = [avg[i] - self.K for i in range(len(avg))]
        c = [np.maximum(c[i], 0) for i in range(len(avg))]
#         c = []
#         for i in range(len(avg)):
#             c.append(avg[i] - self.K)
#             c[i] = np.maximum(c[i], 0)

        # Beregning af payoff for put
        p = [self.K - avg[i] for i in range(len(avg))]
        p = [np.maximum(p[i], 0) for i in range(len(avg))]
#         p = []
#         for i in range(len(avg)):
#             p.append(self.K - avg[i])
#             p[i] = np.maximum(p[i], 0)

        # Gennemsnitligt payoff tilbagediskonteret
        payoff_call = np.mean(c)*np.exp(-self.r*self.T)
        payoff_put = np.mean(p)*np.exp(-self.r*self.T)
    
        return payoff_call, payoff_put

In [32]:
AsianOption = Asian_option_pricing(S0, K, T, r, sigma, n_simulations, n_steps)

In [33]:
AsianOption.MC_Payoff

(6.045392964152429, 3.000455736461755)

In [36]:
def Price_European_Option(S0, K, T, r, sigma, n_simulations, n_steps):
    # Udvikling i prisen på det underliggende aktiv
    dt = T/n_steps
    Z = np.random.normal(0,1,size=(n_simulations, n_steps))
    S = np.zeros((n_simulations, n_steps))
    S[:,0] += S0
    for i in range(1, n_steps):
        S[:,i] += S[:,i-1]*np.exp((r-(sigma**2)/2)*dt+sigma*np.sqrt(dt)*Z[:,i])
    
    # Beregning af payoff for call
    c = S[:,-1] - K
    for i in range(len(c)):
        c[i] = np.maximum(c[i], 0)

    # Beregning af payoff for put
    p = K - S[:,-1]
    for i in range(len(p)):
        p[i] = np.maximum(p[i], 0)

    # Gennemsnitligt payoff tilbagediskonteret
    payoff_call = np.mean(c)*np.exp(-r*T)
    payoff_put = np.mean(p)*np.exp(-r*T)
    
    return payoff_call, payoff_put

In [37]:
Price_European_Option(S0, K, T, r, sigma, n_simulations, n_steps)

(11.53674154365875, 4.757648728444348)

**Tjek med Black-Scholes**

$C_G = S_0 e^{(b-r)T} \Phi(d_1) - K e^{-rT}\Phi(d_2)$

$P_G = K e^{-rT}\Phi(-d_2) - S_0 e^{(b-r)T} \Phi(-d_1)$

$\sigma_G = \frac{\sigma}{\sqrt{3}}$

$b = \frac{1}{2}(r-\frac{1}{2} \sigma_G^2)$

$d_1 = \frac{log(\frac{S_0}{K})+(b+\frac{1}{2}\sigma_G^2)T}{\sigma_G\sqrt{T}}$

$d_2 = d_1 - \sigma_G \sqrt{T}$

In [34]:
def BS_Price_Asian_Options(S0, K, T, r, sigma): 
    sigma_G = sigma/np.sqrt(3)
    b = (1/2)*(r-(1/2)*(sigma_G**2))
    d1 = (np.log(S0/K)+(b+1/2*(sigma_G**2))*T)/(sigma_G*np.sqrt(T))
    d2 = d1 - sigma_G*np.sqrt(T)

    asian_call = S0*np.exp((b-r)*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    asian_put = K*np.exp(-r*T)*norm.cdf(-d2) - S0*np.exp((b-r)*T)*norm.cdf(-d1)
    
    return asian_call, asian_put

In [35]:
BS_Price_Asian_Options(S0, K, T, r, sigma)

(6.024544283554853, 3.024717268581746)

In [105]:
def BS_Price_European_Options(S0, K, T, r, sigma): 
    d1 = (np.log(S0/K)+(r+1/2*(sigma**2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    euro_call = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    euro_put = K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)
    
    return euro_call, euro_put

In [106]:
BS_Price_European_Options(S0, K, T, r, sigma)

(11.541470170672412, 4.780852161267234)

In [59]:
import numpy as np
from scipy.special import erf


class AsianOptionMC_MC(object):
    """ Class for Asian options pricing using control variate
    S0 : float : initial stock/index level
    strike : float : strike price
    T : float : time to maturity (in year fractions)
    M : int : grid or granularity for time (in number of total points)
    r : float : constant risk-free short rate
    div :    float : dividend yield
    sigma :  float : volatility factor in diffusion term 
    
    Unitest (doctest):
    >>> myAsianCall = AsianOptionMC_MC('call', 4., 4., 1., 100., .03, 0, .25, 10000)
    >>> myAsianCall.value
    (0.26141622329842962, 0.25359138249114327, 0.26924106410571597)
    >>> myAsianCall.value_with_control_variate
    (0.25813771282805958, 0.25771678775128265, 0.25855863790483652)
    
    """

    def __init__(self, option_type, S0, strike, T, M, r, div, sigma, simulations):
        try:
            self.option_type = option_type
            assert isinstance(option_type, str)
            self.S0 = float(S0)
            self.strike = float(strike)
            self.T = float(T)
            self.M = int(M)
            self.r = float(r)
            self.div = float(div)
            self.sigma = float(sigma)
            self.simulations = int(simulations)
        except ValueError:
            print('Error passing Options parameters')

        if option_type != 'call' and option_type != 'put':
            raise ValueError("Error: option type not valid. Enter 'call' or 'put'")
        if S0 < 0 or strike < 0 or T <= 0 or r < 0 or div < 0 or sigma < 0:
            raise ValueError('Error: Negative inputs not allowed')

        self.time_unit = self.T / float(self.M)
        self.discount = np.exp(- self.r * self.T)

    @property
    def GeometricAsianOption(self):
        sigsqT = ((self.sigma ** 2 * self.T * (self.M + 1) * (2 * self.M + 1))
                  / (6 * self.M * self.M))
        muT = (0.5 * sigsqT + (self.r - 0.5 * self.sigma ** 2)
               * self.T * (self.M + 1) / (2 * self.M))
        d1 = ((np.log(self.S0 / self.strike) + (muT + 0.5 * sigsqT))
              / np.sqrt(sigsqT))
        d2 = d1 - np.sqrt(sigsqT)
        N1 = 0.5 * (1 + erf(d1 / np.sqrt(2)))
        N2 = 0.5 * (1 + erf(d2 / np.sqrt(2)))
        geometric_value = self.discount * (self.S0 * np.exp(muT) * N1 - self.strike * N2)
        return geometric_value

    @property
    def price_path(self, seed = 100):
        np.random.seed(seed)
        price_path = (self.S0 *
                      np.cumprod (np.exp ((self.r - 0.5 * self.sigma ** 2) * self.time_unit +
                                    self.sigma * np.sqrt(self.time_unit) 
                                          * np.random.randn(self.simulations, self.M)), 1))
        return price_path

    @property
    def MCPayoff(self): 
        if self.option_type == 'call':
            MCpayoff = self.discount \
                              * np.maximum(np.mean(self.price_path,1)-self.strike, 0)
        else:
            MCpayoff = self.discount \
                              * np.maximum(self.strike - np.mean(self.price_path,1), 0)
        return MCpayoff
    
    @property 
    def value(self):
        MCvalue = np.mean(self.MCPayoff)
        MCValue_std = np.std(self.MCPayoff)
        upper_bound = MCvalue + 1.96 * MCValue_std/np.sqrt(self.simulations)
        lower_bound = MCvalue - 1.96 * MCValue_std/np.sqrt(self.simulations)
        return MCvalue, lower_bound, upper_bound
    
    @property
    def value_with_control_variate(self):
        
        geometric_average = np.exp( (1/float(self.M)) * np.sum(np.log(self.price_path), 1) )
        if self.option_type == 'call':
            MCpayoff_geometric = self.discount * np.maximum(geometric_average - self.strike, 0)
        else:
            MCpayoff_geometric = self.discount * np.maximum(self.strike - geometric_average, 0)
        value_with_CV = self.MCPayoff + self.GeometricAsianOption - MCpayoff_geometric        
        value_with_control_variate = np.mean(value_with_CV , 0)
        value_with_control_variate_std = np.std(value_with_CV, 0)
        upper_bound_CV = value_with_control_variate + 1.96 * value_with_control_variate_std/np.sqrt(self.simulations)
        lower_bound_CV = value_with_control_variate - 1.96 * value_with_control_variate_std/np.sqrt(self.simulations)        
        return value_with_control_variate, lower_bound_CV, upper_bound_CV
    


In [62]:
myAsianCall = AsianOptionMC_MC('call', 100, 100, 1., 252, 0.07, 0, 0.2, 100000)

In [63]:
myAsianCall.value

(6.271675559007026, 6.2207727088204114, 6.32257840919364)

In [64]:
myAsianCall.GeometricAsianOption

6.045455740215413